### Investigate the error introduced by the lookup table for the **angle error**

In [6]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm

### Import the `lib` directory
import pathlib
import sys

# To add the files in the lib directory to the importable packages
repo_directory = pathlib.Path().resolve().parents[1]
lib_module_dir = str(repo_directory.joinpath("lib"))
if lib_module_dir not in sys.path:
    sys.path.insert(0, str(repo_directory.joinpath("lib")))

In [7]:
training_dir = ""

assert training_dir != "", (
    "You should pick a directory generated by a model training. These are not uploaded to repository so you need to"
    " generate them yourself by running `angle_error.ipynb`. To save time, you can reduce the number of epochs."
)

### 1. Load the **error model**, i.e., **lookup table**

In [8]:
from error_model import ErrorModel

In [9]:
error_models_path = os.path.join(training_dir, "error_models")

assert os.path.isdir(error_models_path) and os.listdir(error_models_path), "No error models found"

In [10]:
error_model_file = sorted(os.listdir(error_models_path))[-1]
error_model_filepath = os.path.join(error_models_path, error_model_file)
epoch_str = error_model_file.replace("error_model_", "").replace(".pickle", "")

try:
    epoch = int(epoch_str)
except ValueError:
    raise ValueError(f"A valid epoch number could not be obtained for the error model file at {error_model_filepath}")

print(f"Loading the error model from epoch {epoch}...")

with open(error_model_filepath, "rb") as f:
    error_model = pickle.load(f)

print(f"Error model successfully load:")
print(error_model)

Loading the error model from epoch 500...
Error model successfully load:
<class 'error_model.ErrorModel'>
error                    : Lidar range error
param1                   : range
param2                   : angle
ndim                     : 2
x0 boundaries            : (0.1539999991655349, 5.064000129699708)
x1 boundaries            : (1.0023871865882938e-07, 1.2194984821886805)



### 2. Load the **GP**

This is a bit of a hacky solution. Make sure to use the same parameters and data used when training the model.

In [11]:
from dual_gp_model_SVGP import DualGaussianProcessWrapper

2024-09-04 12:54:57.703094: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 12:54:57.743867: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 12:54:57.744658: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 12:54:58.415776: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
params_file = f"epoch {epoch}.pickle"
params_filepath = os.path.join(training_dir, "params", params_file)
assert os.path.isfile(params_filepath), f"Parameters file does not exist for epoch {epoch}. Choose another error model."

In [14]:
SCALE_FACTOR: float = 100

error_data = pd.read_csv("range_error_data.csv")
x_data = np.vstack([error_data["ranges"], error_data["angles"]]).T
y_data = error_data["errors"].to_numpy()

In [15]:
### Use this model only to do predictions
GP_model = DualGaussianProcessWrapper.continue_training(
    x_data=x_data,
    y_data=y_data,
    params_filepath=params_filepath,
    scale_factor=SCALE_FACTOR,
)

Params loaded from Training at 2024-08-15 Thursday at 00.21u/params/epoch 500.pickle...


/home/christiaan/thesis/foresee-error-models/lib/dual_gp_model_SVGP.py:299: UserWarning: Use the model generated by the `continue_training` constructor only for making predictions.
  warnings.warn("Use the model generated by the `continue_training` constructor only for making predictions.")
2024-09-04 12:55:10.935396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-04 12:55:10.937230: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping r

### 3. Find the errors

The biggest error is most likely found at the center of each interpolation interval

In [16]:
def arrays_to_grid_arrays(x0_grid: np.ndarray, x1_grid: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Convert two arrays to two arrays that contain all the points on grid spanned by the two input arrays"""
    x0, x1 = np.meshgrid(x0_grid, x1_grid)
    return tuple(np.vstack((x0.flatten(), x1.flatten())))


def get_test_points(x0_grid: np.ndarray, x1_grid: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Get all test points for a 2D grid. Three positions are tested:
        1) between the points along parameter x0
        2) between the points along parameter x1
        3) between the points along parameter x0 and x1
    """
    x0_grid_between, x1_grid_between = x0_grid[:-1] + np.diff(x0_grid) / 2, x1_grid[:-1] + np.diff(x1_grid) / 2

    x_test_arrays = []
    # between the x0 points
    x_test_arrays.append(arrays_to_grid_arrays(x0_grid_between, x1_grid))
    # between the x1 points
    x_test_arrays.append(arrays_to_grid_arrays(x0_grid, x1_grid_between))
    # between x0 and x1 points
    x_test_arrays.append(arrays_to_grid_arrays(x0_grid_between, x1_grid_between))

    return tuple(np.hstack(x_test_arrays))


def split_up_GP_prediction(GP_model, x_test: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Divide the GP prediction over smaller batches to keep the CPU and memory usage limited."""
    GP_mean_arrays, GP_std_arrays = [], []

    steps = 100
    for i in tqdm(range(steps)):
        x_test_part = np.array_split(x_test, steps)[i]
        values = GP_model.fast_predict_y(x_test_part)
        _GP_mean, _GP_var = [v.numpy().squeeze() for v in values]
        _GP_std = np.sqrt(_GP_var)
        GP_mean_arrays.append(_GP_mean)
        GP_std_arrays.append(_GP_std)

    return np.hstack(GP_mean_arrays), np.hstack(GP_std_arrays)

In [17]:
x0_test, x1_test = get_test_points(*error_model.mean_interpolator.grid)
x_test = np.vstack((x0_test, x1_test)).T

print("Calculating the GP model predictions")
GP_mean, GP_std = split_up_GP_prediction(GP_model, x_test)

print("Calculating the error model predictions")
em_mean, em_std = error_model.mean_interpolator(x_test), error_model.std_interpolator(x_test)

errors_mean = np.abs(GP_mean - em_mean)
errors_std = np.abs(GP_std - em_std)

Calculating the GP model predictions


  0%|          | 0/100 [00:00<?, ?it/s]

Calculating the error model predictions


In [18]:
print("Biggest errors:")
print(f"\tmean error = {errors_mean.max()}")
print(f"\tstd error = {errors_std.max()}")

print("Average values:")
print(f"\tmean avg. = {GP_mean.mean()}")
print(f"\tstd avg. = {GP_std.mean()}")

Biggest errors:
	mean error = 3.242523377161577e-07
	std error = 1.1649695285595452e-06
Average values:
	mean avg. = 0.0362079515702838
	std avg. = 0.007842485748157338
